In [27]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain-core langchain_openai python-dotenv langsmith pydantic spotipy

In [2]:
%pip install --quiet -U jupyterlab-lsp
%pip install --quiet -U "python-lsp-server[all]"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
## Setup logging
import logging
import os
from dotenv import load_dotenv

load_dotenv(override=True)
logger = logging.getLogger(__name__)

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',  # Define the format
    handlers=[logging.StreamHandler()]  # Output to the console
)

# Define Spotify URI type

In [2]:
from typing import List, NewType

# Define a custom URI type
SpotifyURI = NewType('SpotifyURI', str)

In [3]:
from typing import List, Dict, Optional
from pydantic import BaseModel, Field

# Define a Pydantic model for Playlist
class Playlist(BaseModel):
    """
    A model representing a Spotify playlist.
    """
    id: str = Field(..., description="The unique identifier for the playlist.")
    uri: str = Field(..., description="The Spotify URI for the playlist.")
    name: str = Field(..., description="The name of the playlist.")
    description: Optional[str] = Field(None, description="The playlist's description.")
    owner: Optional[str] = Field(None, description="The display name of the playlist's owner.")
    tracks_total: Optional[int] = Field(None, description="The total number of tracks in the playlist.")
    is_public: Optional[bool] = Field(None, description="Indicates if the playlist is public.")
    collaborative: Optional[bool] = Field(None, description="Indicates if the playlist is collaborative.")
    snapshot_id: Optional[str] = Field(None, description="The version identifier for the current playlist.")

# Define a Pydantic model for Track
class Track(BaseModel):
    """
    A model representing a Spotify track.
    """
    id: str = Field(..., description="The unique identifier for the track.")
    uri: str = Field(..., description="The Spotify URI for the track.")
    name: str = Field(..., description="The name of the track.")
    artists: List[str] = Field(..., description="A list of artists who performed the track.")
    album: str = Field(..., description="The name of the album the track is from.")
    duration_ms: Optional[int] = Field(None, description="The track length in milliseconds.")
    explicit: Optional[bool] = Field(None, description="Indicates if the track has explicit content.")
    popularity: Optional[int] = Field(None, description="The popularity of the track (0-100).")

In [4]:
from typing import Dict, List, Any, Annotated
from typing import Annotated
from typing_extensions import TypedDict
from langchain_core.messages import AIMessage, HumanMessage, BaseMessage, ToolMessage
from langgraph.graph.message import add_messages


class State(TypedDict):
    """
    Represents the state of the conversation and Spotify information

    Attributes:
        playlists (List[Playlist]): A list of Spotify Playlists.
        tracks (List[Track]): Track list for a Spotify Playlist
        new_playlist: (Playlist) : New Spotify playlist data
        new_tracks: (List[Track]): Tracks for the new playlist
    """
    new_playlist: Playlist
    new_tracks: List[Track]
    playlists: List[Playlist]
    tracks: List[Track]
    messages: Annotated[List[BaseMessage], add_messages]


# Define Spotify Tools

In [ ]:
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from langchain_core.tools import tool
from typing import List, NewType
import json


def get_spotify_client() -> spotipy.Spotify:
    """
    Initializes and returns a Spotify client with user authentication.

    Returns:
        spotipy.Spotify: An authenticated Spotify client.
    """
    auth_manager = SpotifyClientCredentials(
        client_id=os.environ.get("SPOTIFY_CLIENT_ID"),
        client_secret=os.environ.get("SPOTIFY_CLIENT_SECRET"),
    )
    return spotipy.Spotify(auth_manager=auth_manager)


@tool
def get_playlists() -> Dict[str, List[Playlist]]:
    """
    Retrieves a list of the user's Spotify playlists.

    Returns:
        Dict[str, List[Playlist]]: A dictionary containing a list of Playlist models under the 'playlists' key.
    """
    sp = get_spotify_client()
    playlists: List[Playlist] = []

    try:
        # Fetch the current user's playlists with pagination
        playlists_raw = sp.user_playlists(user=os.getenv("SPOTIFY_USER_ID"), limit=100)
        while playlists_raw:
            for playlist_data in playlists_raw['items']:
                # Map API data to the Playlist model
                playlist = Playlist(
                    id=playlist_data['id'],
                    uri=playlist_data['uri'],
                    name=playlist_data['name'],
                    description=playlist_data.get('description'),
                    owner=playlist_data['owner']['display_name'],
                    tracks_total=playlist_data['tracks']['total'],
                    is_public=playlist_data.get('public'),
                    collaborative=playlist_data.get('collaborative'),
                    snapshot_id=playlist_data.get('snapshot_id')
                )
                playlists.append(playlist)
            # Check if there is a next page
            if playlists_raw['next']:
                playlists_raw = sp.next(playlists_raw)
            else:
                break
    except spotipy.SpotifyException as e:
        return {"error": str(e)}

    # Serialize the playlists to JSON-serializable dictionaries
    serialized_playlists = [playlist.model_dump() for playlist in playlists]
    return {"playlists": serialized_playlists}


@tool
def get_track_list(playlist_id: str) -> Dict[str, List[Track]]:
    """
    Retrieves the track list for a specific Spotify playlist.

    Args:
        playlist_id (str): The unique identifier of the Spotify playlist.

    Returns:
        Dict[str, List[Track]]: A dictionary containing a list of Track models under the 'tracks' key.
    """
    sp = get_spotify_client()
    playlist_tracks: List[Track] = []

    try:
        # Fetch the playlist's tracks with pagination
        playlist = sp.user_playlist(user=os.getenv("SPOTIFY_USER_ID"), playlist_id=playlist_id)
        if 'tracks' in playlist:
            tracks = playlist["tracks"]
            while tracks:
                for item in tracks['items']:
                    track_data = item['track']
                    # Map API data to the Track model
                    track = Track(
                        id=track_data['id'],
                        uri=track_data['uri'],
                        name=track_data['name'],
                        artists=[artist['name'] for artist in track_data['artists']],
                        album=track_data['album']['name'],
                        duration_ms=track_data.get('duration_ms'),
                        explicit=track_data.get('explicit'),
                        popularity=track_data.get('popularity')
                    )
                    playlist_tracks.append(track)
                # Check if there is a next page
                if tracks['next']:
                    # TODO unclear in this case
                    tracks = sp.next(tracks)
                else:
                    break
    except spotipy.SpotifyException as e:
        return {"error": str(e)}

    # Serialize the tracks to JSON-serializable dictionaries
    serialized_tracks = [track.model_dump() for track in tracks]
    return {"tracks": serialized_tracks}


@tool
def create_spotify_playlist(name: str, description: str = "Agentic Playlist") -> Dict[str, Playlist]:
    """
    Creates a new playlist on Spotify.

    This function only creates the playlist; it does not add tracks.

    Args:
        name (str): The name of the new playlist.
        description (str, optional): The description of the playlist.

    Returns:
        Dict[str, Playlist]: A dictionary containing the new Playlist model under the 'new_playlist' key.
    """
    sp = get_spotify_client()
    try:
        # Retrieve the current user's ID
        user_id = sp.current_user()['id']
        # Create a new playlist
        new_playlist_data = sp.user_playlist_create(
            user=user_id,
            name=name,
            public=True,
            description=description
        )
        # Map API data to the Playlist model
        new_playlist = Playlist(
            id=new_playlist_data['id'],
            uri=new_playlist_data['uri'],
            name=new_playlist_data['name'],
            description=new_playlist_data.get('description'),
            owner=new_playlist_data['owner']['display_name'],
            tracks_total=new_playlist_data['tracks']['total'],
            is_public=new_playlist_data.get('public'),
            collaborative=new_playlist_data.get('collaborative'),
            snapshot_id=new_playlist_data.get('snapshot_id')
        )
    except spotipy.SpotifyException as e:
        return {"error": str(e)}
    return {"new_playlist": new_playlist.model_dump()}

@tool
def add_tracks_to_playlist(playlist_id: str, tracks: List[SpotifyURI]) -> Dict[str, Any]:
    """
    Adds tracks to a Spotify playlist.

    Args:
        playlist_id (str): ID of the playlist.
        tracks (List[SpotifyURI]): List of Spotify track URIs.

    Returns:
        Dict[str, Any]: A dictionary indicating success or error.
    """
    sp = get_spotify_client()
    try:
        sp.playlist_add_items(playlist_id=playlist_id, items=tracks)
    except spotipy.SpotifyException as e:
        return {"error": str(e)}
    return {"success": True} 



# Define Search Tool

In [6]:
from langchain_community.tools.tavily_search import TavilySearchResults

search_tool = TavilySearchResults(
    max_results=5,
    include_answer=True,
    include_raw_content=True,
    include_images=True,
    # search_depth="advanced",
    # include_domains = []
    # exclude_domains = []
)
name = search_tool.get_name()
desc = search_tool.description
desc

'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.'

# Create ToolNode

In [7]:
from langgraph.prebuilt import ToolNode

tools = [get_playlists, get_track_list, create_spotify_playlist, add_tracks_to_playlist, search_tool]
tool_node = ToolNode(tools)

# Manual Test of Tool Infra

In [164]:
from langchain_core.messages import AIMessage, HumanMessage
message_with_single_tool_call = AIMessage(
    content="",
    tool_calls=[
        {
            "name": "get_playlists",
            "args" : {},
            "id": "tool_call_id",
            "type": "tool_call",
        }
    ],
)

tool_node.invoke({"messages": [message_with_single_tool_call]})

{'messages': [ToolMessage(content='{"playlists": ["1, spotify:playlist:2dbYK5b7J0F7IdH5n1TEUK,  RPreacher", "2, spotify:playlist:4ack9YtUhdxRayJDAqlfQe,  RP Bossa Nova Chill ", "3, spotify:playlist:75NW18NgdZuZeifrcjxKlZ,  GVF", "4, spotify:playlist:5iyONtUO21O88xw8pBblwh,  Now And Then", "5, spotify:playlist:4SDSUMg2HJcGFmHlupsU7U,  Jazz classics", "6, spotify:playlist:5fBxLG2wLAPVpx83402rHl,  Acdc", "7, spotify:playlist:17Yxm5hQbkbZwDcCTuiDXC,  einaudi", "8, spotify:playlist:2X6GccWfIqPWO6VYx405nO,  Eric Clapton Blues", "9, spotify:playlist:3smVXe8y8w8nt4lFH0HBkW,  Alchemy: Dire Straits", "10, spotify:playlist:2ifRZBXnX7UmIPUlGet5WZ,  Blues", "11, spotify:playlist:5nn2ePtd9gYEEDL3XnACCB,  brazil", "12, spotify:playlist:6YAntoKNXuogPKdV26b6XE,  CCR", "13, spotify:playlist:2VHkZvOMCkLF3B7TsKerKD,  Chill", "14, spotify:playlist:508lbLx9XUkHv5Xb45hxTP,  Classic Rock", "15, spotify:playlist:7zzDMq1sPwkZPYeqOQJ9qS,  The New Four Seasons - Vivaldi Recomposed", "16, spotify:playlist:4l8WcaYl

# Bind Tools to Model

In [8]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=os.getenv("OPENAI_MODEL_NAME"))
llm_with_tools = llm.bind_tools(tools)

# Define Prompt

In [9]:
prompt = """
Create and execute a step-by-step plan with atomic tasks to solve the following problem:

Build a Spotify Playlist with tracks of the same vibe as my 'New Rock' playlist. Here are some rules

1 - Only suggest tracks from artist not present on the 'New Rock' playlist
2 - Artist should have achieved success after year 2010
3 - Do not use 'The Raconteurs' as an artist in the new playlist
4 - Playlist should have around 100 tracks
5 - Arrange the tracks to create a smooth listening experience, considering tempo, energy, and mood.
6 - Add some hidden gems
7 - Use your knowledge as a recommendation music system. Think carefully and thoroughly. 
"""

# First Message

In [10]:
from langchain_core.messages import AIMessage, HumanMessage

human_message = HumanMessage(prompt)
ai_tool_call_message = llm_with_tools.invoke([human_message])
ai_tool_call_message
#print(f"tool_call_message: {tool_call_message.pretty_print()}")


2024-11-11 18:52:41,471 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


AIMessage(content='To create a Spotify playlist that matches the vibe of your "New Rock" playlist while adhering to the given rules, here\'s a step-by-step plan with atomic tasks:\n\n### Step 1: Gather Information\n1. **Retrieve Existing Playlist Details**\n   - Fetch the list of your Spotify playlists to identify the "New Rock" playlist.\n   - Retrieve the track list from the "New Rock" playlist to understand its vibe and collect data on the artists featured.\n\n### Step 2: Research and Analyze\n2. **Identify Similar Artists**\n   - Search for artists similar to those in the "New Rock" playlist who achieved success after 2010, excluding "The Raconteurs."\n\n3. **Find Hidden Gems**\n   - Research lesser-known tracks or upcoming artists that fit the vibe of the "New Rock" playlist and have gained some recognition post-2010.\n\n### Step 3: Curate New Playlist\n4. **Select Tracks**\n   - Choose around 100 tracks from the identified artists, ensuring no artists from the "New Rock" playlist

In [11]:
print(ai_tool_call_message.content)

To create a Spotify playlist that matches the vibe of your "New Rock" playlist while adhering to the given rules, here's a step-by-step plan with atomic tasks:

### Step 1: Gather Information
1. **Retrieve Existing Playlist Details**
   - Fetch the list of your Spotify playlists to identify the "New Rock" playlist.
   - Retrieve the track list from the "New Rock" playlist to understand its vibe and collect data on the artists featured.

### Step 2: Research and Analyze
2. **Identify Similar Artists**
   - Search for artists similar to those in the "New Rock" playlist who achieved success after 2010, excluding "The Raconteurs."

3. **Find Hidden Gems**
   - Research lesser-known tracks or upcoming artists that fit the vibe of the "New Rock" playlist and have gained some recognition post-2010.

### Step 3: Curate New Playlist
4. **Select Tracks**
   - Choose around 100 tracks from the identified artists, ensuring no artists from the "New Rock" playlist are repeated.
   - Include hidden g

# chat_prompt_template holds all the messages (Human, AI, Tool)

In [12]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt_template: ChatPromptTemplate = human_message + ai_tool_call_message
chat_prompt_template.format_messages()

[HumanMessage(content="\nCreate and execute a step-by-step plan with atomic tasks to solve the following problem:\n\nBuild a Spotify Playlist with tracks of the same vibe as my 'New Rock' playlist. Here are some rules\n\n1 - Only suggest tracks from artist not present on the 'New Rock' playlist\n2 - Artist should have achieved success after year 2010\n3 - Do not use 'The Raconteurs' as an artist in the new playlist\n4 - Playlist should have around 100 tracks\n5 - Arrange the tracks to create a smooth listening experience, considering tempo, energy, and mood.\n6 - Add some hidden gems\n7 - Use your knowledge as a recommendation music system. Think carefully and thoroughly. \n", additional_kwargs={}, response_metadata={}),
 AIMessage(content='To create a Spotify playlist that matches the vibe of your "New Rock" playlist while adhering to the given rules, here\'s a step-by-step plan with atomic tasks:\n\n### Step 1: Gather Information\n1. **Retrieve Existing Playlist Details**\n   - Fetch

# Tool Call (get_playlists)

In [13]:
import json
from langchain_core.messages import ToolMessage

response = tool_node.invoke({"messages": [ai_tool_call_message]})
response


{'messages': [ToolMessage(content='{"playlists": [{"id": "2dbYK5b7J0F7IdH5n1TEUK", "uri": "spotify:playlist:2dbYK5b7J0F7IdH5n1TEUK", "name": "RPreacher", "description": "", "owner": "Ray", "tracks_total": 3, "is_public": true, "collaborative": false, "snapshot_id": "AAAABG1lhIiRJT/NE9RpP4K6UnmZWixo"}, {"id": "4ack9YtUhdxRayJDAqlfQe", "uri": "spotify:playlist:4ack9YtUhdxRayJDAqlfQe", "name": "RP Bossa Nova Chill ", "description": "", "owner": "Ray", "tracks_total": 30, "is_public": true, "collaborative": false, "snapshot_id": "AAAAIThdM9g/YPQwj7nuoHhewWyLoKEi"}, {"id": "75NW18NgdZuZeifrcjxKlZ", "uri": "spotify:playlist:75NW18NgdZuZeifrcjxKlZ", "name": "GVF", "description": "", "owner": "Ray", "tracks_total": 11, "is_public": true, "collaborative": false, "snapshot_id": "AAAADKZNV9h+cfi1PbOi6yjNHMe8mLWC"}, {"id": "5iyONtUO21O88xw8pBblwh", "uri": "spotify:playlist:5iyONtUO21O88xw8pBblwh", "name": "Now And Then", "description": "", "owner": "Ray", "tracks_total": 7, "is_public": true, "col

In [14]:
message_list: List[ToolMessage] = response["messages"]
playlist_tool_message: ToolMessage = message_list[0]
playlist_tool_message

ToolMessage(content='{"playlists": [{"id": "2dbYK5b7J0F7IdH5n1TEUK", "uri": "spotify:playlist:2dbYK5b7J0F7IdH5n1TEUK", "name": "RPreacher", "description": "", "owner": "Ray", "tracks_total": 3, "is_public": true, "collaborative": false, "snapshot_id": "AAAABG1lhIiRJT/NE9RpP4K6UnmZWixo"}, {"id": "4ack9YtUhdxRayJDAqlfQe", "uri": "spotify:playlist:4ack9YtUhdxRayJDAqlfQe", "name": "RP Bossa Nova Chill ", "description": "", "owner": "Ray", "tracks_total": 30, "is_public": true, "collaborative": false, "snapshot_id": "AAAAIThdM9g/YPQwj7nuoHhewWyLoKEi"}, {"id": "75NW18NgdZuZeifrcjxKlZ", "uri": "spotify:playlist:75NW18NgdZuZeifrcjxKlZ", "name": "GVF", "description": "", "owner": "Ray", "tracks_total": 11, "is_public": true, "collaborative": false, "snapshot_id": "AAAADKZNV9h+cfi1PbOi6yjNHMe8mLWC"}, {"id": "5iyONtUO21O88xw8pBblwh", "uri": "spotify:playlist:5iyONtUO21O88xw8pBblwh", "name": "Now And Then", "description": "", "owner": "Ray", "tracks_total": 7, "is_public": true, "collaborative": f

In [15]:
json.loads(playlist_tool_message.content)

{'playlists': [{'id': '2dbYK5b7J0F7IdH5n1TEUK',
   'uri': 'spotify:playlist:2dbYK5b7J0F7IdH5n1TEUK',
   'name': 'RPreacher',
   'description': '',
   'owner': 'Ray',
   'tracks_total': 3,
   'is_public': True,
   'collaborative': False,
   'snapshot_id': 'AAAABG1lhIiRJT/NE9RpP4K6UnmZWixo'},
  {'id': '4ack9YtUhdxRayJDAqlfQe',
   'uri': 'spotify:playlist:4ack9YtUhdxRayJDAqlfQe',
   'name': 'RP Bossa Nova Chill ',
   'description': '',
   'owner': 'Ray',
   'tracks_total': 30,
   'is_public': True,
   'collaborative': False,
   'snapshot_id': 'AAAAIThdM9g/YPQwj7nuoHhewWyLoKEi'},
  {'id': '75NW18NgdZuZeifrcjxKlZ',
   'uri': 'spotify:playlist:75NW18NgdZuZeifrcjxKlZ',
   'name': 'GVF',
   'description': '',
   'owner': 'Ray',
   'tracks_total': 11,
   'is_public': True,
   'collaborative': False,
   'snapshot_id': 'AAAADKZNV9h+cfi1PbOi6yjNHMe8mLWC'},
  {'id': '5iyONtUO21O88xw8pBblwh',
   'uri': 'spotify:playlist:5iyONtUO21O88xw8pBblwh',
   'name': 'Now And Then',
   'description': '',
   'ow

In [16]:
# We recreate the ToolMessage to take care of UNicode characters
tool_message = ToolMessage(content=playlist_tool_message.content, name=playlist_tool_message.name, tool_call_id=playlist_tool_message.tool_call_id)
tool_message

ToolMessage(content='{"playlists": [{"id": "2dbYK5b7J0F7IdH5n1TEUK", "uri": "spotify:playlist:2dbYK5b7J0F7IdH5n1TEUK", "name": "RPreacher", "description": "", "owner": "Ray", "tracks_total": 3, "is_public": true, "collaborative": false, "snapshot_id": "AAAABG1lhIiRJT/NE9RpP4K6UnmZWixo"}, {"id": "4ack9YtUhdxRayJDAqlfQe", "uri": "spotify:playlist:4ack9YtUhdxRayJDAqlfQe", "name": "RP Bossa Nova Chill ", "description": "", "owner": "Ray", "tracks_total": 30, "is_public": true, "collaborative": false, "snapshot_id": "AAAAIThdM9g/YPQwj7nuoHhewWyLoKEi"}, {"id": "75NW18NgdZuZeifrcjxKlZ", "uri": "spotify:playlist:75NW18NgdZuZeifrcjxKlZ", "name": "GVF", "description": "", "owner": "Ray", "tracks_total": 11, "is_public": true, "collaborative": false, "snapshot_id": "AAAADKZNV9h+cfi1PbOi6yjNHMe8mLWC"}, {"id": "5iyONtUO21O88xw8pBblwh", "uri": "spotify:playlist:5iyONtUO21O88xw8pBblwh", "name": "Now And Then", "description": "", "owner": "Ray", "tracks_total": 7, "is_public": true, "collaborative": f

# Add To message template list

In [17]:
chat_prompt_template += tool_message

# Format Message List for LLM

In [18]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt_template.format_messages()

[HumanMessage(content="\nCreate and execute a step-by-step plan with atomic tasks to solve the following problem:\n\nBuild a Spotify Playlist with tracks of the same vibe as my 'New Rock' playlist. Here are some rules\n\n1 - Only suggest tracks from artist not present on the 'New Rock' playlist\n2 - Artist should have achieved success after year 2010\n3 - Do not use 'The Raconteurs' as an artist in the new playlist\n4 - Playlist should have around 100 tracks\n5 - Arrange the tracks to create a smooth listening experience, considering tempo, energy, and mood.\n6 - Add some hidden gems\n7 - Use your knowledge as a recommendation music system. Think carefully and thoroughly. \n", additional_kwargs={}, response_metadata={}),
 AIMessage(content='To create a Spotify playlist that matches the vibe of your "New Rock" playlist while adhering to the given rules, here\'s a step-by-step plan with atomic tasks:\n\n### Step 1: Gather Information\n1. **Retrieve Existing Playlist Details**\n   - Fetch

# Send Playlists Results to LLM

In [19]:
ai_tool_call_message_2 = llm_with_tools.invoke(chat_prompt_template.format_messages())

2024-11-11 18:53:06,978 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [20]:
chat_prompt_template += ai_tool_call_message_2
chat_prompt_template.format_messages()

[HumanMessage(content="\nCreate and execute a step-by-step plan with atomic tasks to solve the following problem:\n\nBuild a Spotify Playlist with tracks of the same vibe as my 'New Rock' playlist. Here are some rules\n\n1 - Only suggest tracks from artist not present on the 'New Rock' playlist\n2 - Artist should have achieved success after year 2010\n3 - Do not use 'The Raconteurs' as an artist in the new playlist\n4 - Playlist should have around 100 tracks\n5 - Arrange the tracks to create a smooth listening experience, considering tempo, energy, and mood.\n6 - Add some hidden gems\n7 - Use your knowledge as a recommendation music system. Think carefully and thoroughly. \n", additional_kwargs={}, response_metadata={}),
 AIMessage(content='To create a Spotify playlist that matches the vibe of your "New Rock" playlist while adhering to the given rules, here\'s a step-by-step plan with atomic tasks:\n\n### Step 1: Gather Information\n1. **Retrieve Existing Playlist Details**\n   - Fetch

# Tool Call (get_track_list)

In [ ]:
response = tool_node.invoke({"messages": [ai_tool_call_message_2]})
response

# Decode AI Tool Call message

In [173]:
track_list_tool_message = response["messages"][0]

In [174]:
content = json.loads(track_list_tool_message.content)
tracks_raw = content["tracks"]["items"]   # full raw spotify tracks data

# Track List Model

In [175]:
from pydantic import BaseModel, Field
from typing import List, Optional

class Track(BaseModel):
    """This class represents a single track"""
    artist: str = Field(..., description="The artist of the track")
    name: str = Field(..., description="The name of the track")

class Tracks(BaseModel):
    """This class contains the track list for a Spotify playlist"""
    tracks: Optional[List[Track]] = Field(None, description="A list of Track objects representing the playlist tracks")


In [176]:
# Display track name and artist in the format <trackname>:<artist>

track_list = []

for item in tracks_raw:
    track = item['track']
    track_name = track['name']
    artist_name = track['artists'][0]['name']
    track = Track(artist=artist_name, name=track_name)
    track_list.append(track)  # Taking only the first artist
    # print(f"{track_name}:{artist_name}")

tracks = Tracks(tracks=track_list)
# print(tracks.model_dump_json(indent=2))

# Create Track List ToolMessage for LLM

In [177]:
tool_message = ToolMessage(content=tracks.model_dump_json(indent=2), name=track_list_tool_message.name, tool_call_id=track_list_tool_message.tool_call_id)

# Add Tool message containing playlist tracks to list of messages

In [178]:
chat_prompt_template += tool_message

In [179]:
chat_prompt_template.format_messages()

[HumanMessage(content="\nBuild a Spotify Playlist with tracks of the same vibe as my 'New Rock' playlist. Here are some rules\n\n1 - Only suggest tracks from artist not present on the 'New Rock' playlist\n2 - Artist should have achieved success after year 2010\n3 - Do not use 'The Raconteurs' as an artist\n4 - Playlist should have around 100 tracks\n5 - Feel free to add some hidden gems\n", additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_qMkFqJ5eP6s5pZ9VdtCM2ZOw', 'function': {'arguments': '{}', 'name': 'get_playlists'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 222, 'total_tokens': 233, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 

# Send Tool Result to LLM

In [180]:
llm_with_structured_output = llm.with_structured_output(schema=Tracks, method='json_schema')
response = llm_with_structured_output.invoke(chat_prompt_template.format_messages())
# response = llm_with_tools.invoke(chat_prompt_template.format_messages())
response

2024-11-11 02:08:15,426 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Tracks(tracks=[Track(artist='Royal Blood', name='Figure It Out'), Track(artist='The Struts', name='Could Have Been Me'), Track(artist='Highly Suspect', name='My Name Is Human'), Track(artist='Nothing But Thieves', name='Amsterdam'), Track(artist='Rival Sons', name='Pressure and Time'), Track(artist='The Amazons', name='In My Mind'), Track(artist='Shame', name='Concrete'), Track(artist='Wolf Alice', name='Giant Peach'), Track(artist='IDLES', name='Danny Nedelko'), Track(artist='Fontaines D.C.', name='Boys in the Better Land'), Track(artist='Black Honey', name='Corrine'), Track(artist='The Vaccines', name="I Can't Quit"), Track(artist='Courtney Barnett', name='Pedestrian at Best'), Track(artist='Fidlar', name='40oz. On Repeat'), Track(artist='King Gizzard & The Lizard Wizard', name='Rattlesnake'), Track(artist='The Mysterines', name="Love's Not Enough"), Track(artist='The Nude Party', name='Chevrolet Van'), Track(artist='Greta Van Fleet', name='Safari Song'), Track(artist='Sam Fender', n

In [181]:
response.tracks

[Track(artist='Royal Blood', name='Figure It Out'),
 Track(artist='The Struts', name='Could Have Been Me'),
 Track(artist='Highly Suspect', name='My Name Is Human'),
 Track(artist='Nothing But Thieves', name='Amsterdam'),
 Track(artist='Rival Sons', name='Pressure and Time'),
 Track(artist='The Amazons', name='In My Mind'),
 Track(artist='Shame', name='Concrete'),
 Track(artist='Wolf Alice', name='Giant Peach'),
 Track(artist='IDLES', name='Danny Nedelko'),
 Track(artist='Fontaines D.C.', name='Boys in the Better Land'),
 Track(artist='Black Honey', name='Corrine'),
 Track(artist='The Vaccines', name="I Can't Quit"),
 Track(artist='Courtney Barnett', name='Pedestrian at Best'),
 Track(artist='Fidlar', name='40oz. On Repeat'),
 Track(artist='King Gizzard & The Lizard Wizard', name='Rattlesnake'),
 Track(artist='The Mysterines', name="Love's Not Enough"),
 Track(artist='The Nude Party', name='Chevrolet Van'),
 Track(artist='Greta Van Fleet', name='Safari Song'),
 Track(artist='Sam Fender

In [182]:
chat_prompt_template += HumanMessage("You have reused artists from the New Rock playlist, try again")
llm_with_structured_output = llm.with_structured_output(schema=Tracks, method='json_schema')
response = llm_with_structured_output.invoke(chat_prompt_template.format_messages())
response

2024-11-11 02:08:37,499 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Tracks(tracks=[Track(artist='Royal Blood', name="Trouble's Coming"), Track(artist='IDLES', name='Danny Nedelko'), Track(artist='Fontaines D.C.', name='Boys in the Better Land'), Track(artist='The Amazons', name='Mother'), Track(artist='Black Midi', name='John L'), Track(artist='King Gizzard & The Lizard Wizard', name='Self-Immolate'), Track(artist='Wolf Alice', name='Smile'), Track(artist='Biffy Clyro', name='Space'), Track(artist='The War on Drugs', name='Pain'), Track(artist='The Mysterines', name='In My Head'), Track(artist='Shame', name='Concrete'), Track(artist='Viagra Boys', name='Sports'), Track(artist='Sleaford Mods', name='Nudge It (feat. Amy Taylor)'), Track(artist='Parquet Courts', name='Wide Awake'), Track(artist='Courtney Barnett', name='Pedestrian at Best'), Track(artist='Tash Sultana', name='Jungle'), Track(artist='The Nude Party', name='Chevrolet Van'), Track(artist='Phoebe Bridgers', name='Kyoto'), Track(artist='Sam Fender', name='Seventeen Going Under'), Track(artist=

In [184]:
response.tracks

[Track(artist='Royal Blood', name="Trouble's Coming"),
 Track(artist='IDLES', name='Danny Nedelko'),
 Track(artist='Fontaines D.C.', name='Boys in the Better Land'),
 Track(artist='The Amazons', name='Mother'),
 Track(artist='Black Midi', name='John L'),
 Track(artist='King Gizzard & The Lizard Wizard', name='Self-Immolate'),
 Track(artist='Wolf Alice', name='Smile'),
 Track(artist='Biffy Clyro', name='Space'),
 Track(artist='The War on Drugs', name='Pain'),
 Track(artist='The Mysterines', name='In My Head'),
 Track(artist='Shame', name='Concrete'),
 Track(artist='Viagra Boys', name='Sports'),
 Track(artist='Sleaford Mods', name='Nudge It (feat. Amy Taylor)'),
 Track(artist='Parquet Courts', name='Wide Awake'),
 Track(artist='Courtney Barnett', name='Pedestrian at Best'),
 Track(artist='Tash Sultana', name='Jungle'),
 Track(artist='The Nude Party', name='Chevrolet Van'),
 Track(artist='Phoebe Bridgers', name='Kyoto'),
 Track(artist='Sam Fender', name='Seventeen Going Under'),
 Track(ar